In [591]:
# Paul Corcoran's Code

In [592]:
# Import the relevant packages
import pandas as pd
import requests
from sklearn.preprocessing import MinMaxScaler
import warnings
import json
warnings.filterwarnings('ignore')

In [593]:
# The Fantasy Premier League API provides data which is relevant to the game
# Connect to the API
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
# Make the request
res = requests.get(url)

In [594]:
# Check the status code and print a message
if res.status_code == 200:
    print('The API request was successful.')
else:
    print(f'Error: The API request failed with status code {res.status_code}.')
# The code above should return the text “The API request was successful” if the connection made was a success

The API request was successful.


In [595]:
# Store the output in a json format (typically a dictionary)
json = res.json()

In [596]:
# The .keys() method is a built-in Python function - returns the keys of the dictionary 
json.keys()
# We can see that there are numerous fields within this API

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [597]:
# We are interested in the following fields which we will turn into a pd.DataFrame
# teams (dataframe contains the name and id of each team in the current season) 
# elements_types_df (contains the mapping for player positions)
# elements_df (contain all of the player stats) 

elements_df = pd.DataFrame(json['elements'])
elements_types_df = pd.DataFrame(json['element_types'])
teams_df = pd.DataFrame(json['teams'])

In [598]:
# Anther way to do this
# Grab the data behind the FPL site
# data = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').json()
# Convert the data to a table
# dataTable = pd.DataFrame(data['elements'])
# List the variables
# dataTable.columns
# Select a few columns and show the top ten players by xG per 90
# dataTable[['web_name', 'points_per_game', 'total_points']].sort_values('points_per_game', ascending=False).head(10)

In [599]:
# Back to Paul Corcoran's code

In [600]:
# Have a look at the variables available in elements_df
# These are the various player statistics where are available to you

elements_df.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next',
       'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam',
       'news', 'news_added', 'now_cost', 'photo', 'points_per_game',
       'second_name', 'selected_by_percent', 'special', 'squad_number',
       'status', 'team', 'team_code', 'total_points', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored',
       'assists', 'clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards',
       'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat',
       'ict_index', 'starts', 'expected_goals', 'expected_assists',
       'expected_goal_involvements', 'expected_goals_con

In [601]:
# In order to avoid confusion we rename elements_df as players_df and select the variables that you are interested in

players_df = elements_df[
    ['web_name', 'first_name', 'second_name', 'team', 'element_type', 'now_cost', 'total_points', 'bonus', 'points_per_game', 'form',   
     'minutes', 'starts', 'starts_per_90', 'chance_of_playing_next_round', 'chance_of_playing_this_round', 'selected_by_percent', 
     'transfers_in', 'goals_scored', 'expected_goals', 'expected_goals_per_90', 'assists', 'expected_assists', 'expected_assists_per_90', 
     'expected_goal_involvements', 'expected_goal_involvements_per_90', 'goals_conceded', 'goals_conceded_per_90',
     'expected_goals_conceded', 'expected_goals_conceded_per_90', 'clean_sheets', 'clean_sheets_per_90', 'saves', 'saves_per_90', 
     'penalties_saved']]

# Some columns need to be converted from a string to a number (float)

cols_to_convert = ['now_cost', 'total_points', 'bonus', 'points_per_game', 'form', 'minutes', 'starts', 'starts_per_90', 
                   'selected_by_percent', 'transfers_in', 'goals_scored', 'expected_goals', 'expected_goals_per_90', 'assists', 
                   'expected_assists', 'expected_assists_per_90', 'expected_goal_involvements', 'expected_goal_involvements_per_90', 
                   'goals_conceded', 'goals_conceded_per_90', 'expected_goals_conceded', 'expected_goals_conceded_per_90', 
                   'clean_sheets', 'clean_sheets_per_90', 'saves', 'saves_per_90', 'penalties_saved']

players_df[cols_to_convert] = players_df[cols_to_convert].astype(float)

# Concat the first and second name variables into a new variable (Player)

players_df['Player'] = players_df['first_name'] + ' ' + players_df['second_name']


In [602]:
players_df.columns

Index(['web_name', 'first_name', 'second_name', 'team', 'element_type',
       'now_cost', 'total_points', 'bonus', 'points_per_game', 'form',
       'minutes', 'starts', 'starts_per_90', 'chance_of_playing_next_round',
       'chance_of_playing_this_round', 'selected_by_percent', 'transfers_in',
       'goals_scored', 'expected_goals', 'expected_goals_per_90', 'assists',
       'expected_assists', 'expected_assists_per_90',
       'expected_goal_involvements', 'expected_goal_involvements_per_90',
       'goals_conceded', 'goals_conceded_per_90', 'expected_goals_conceded',
       'expected_goals_conceded_per_90', 'clean_sheets', 'clean_sheets_per_90',
       'saves', 'saves_per_90', 'penalties_saved', 'Player'],
      dtype='object')

In [603]:
players_df

,web_name,first_name,second_name,team,element_type,now_cost,total_points,bonus,points_per_game,form,...,goals_conceded,goals_conceded_per_90,expected_goals_conceded,expected_goals_conceded_per_90,clean_sheets,clean_sheets_per_90,saves,saves_per_90,penalties_saved,Player
0,Balogun,Folarin,Balogun,1,4,45.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,Folarin Balogun
1,Cédric,Cédric,Alves Soares,1,2,40.0,10.0,0.0,1.2,0.0,...,3.0,1.21,2.83,1.14,0.0,0.00,0.0,0.0,0.0,Cédric Alves Soares
2,M.Elneny,Mohamed,Elneny,1,3,45.0,6.0,0.0,1.2,0.0,...,2.0,1.62,1.29,1.05,0.0,0.00,0.0,0.0,0.0,Mohamed Elneny
3,Fábio Vieira,Fábio,Ferreira Vieira,1,3,55.0,40.0,2.0,1.8,0.0,...,5.0,0.90,5.28,0.95,2.0,0.36,0.0,0.0,0.0,Fábio Ferreira Vieira
4,Gabriel,Gabriel,dos Santos Magalhães,1,2,50.0,146.0,15.0,3.8,0.0,...,43.0,1.14,41.84,1.10,14.0,0.37,0.0,0.0,0.0,Gabriel dos Santos Magalhães
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,N.Semedo,Nélson,Cabral Semedo,20,2,45.0,75.0,5.0,2.1,0.0,...,43.0,1.47,50.02,1.71,8.0,0.27,0.0,0.0,0.0,Nélson Cabral Semedo
594,Toti,Toti António,Gomes,20,2,45.0,43.0,4.0,2.5,0.0,...,14.0,1.29,20.87,1.92,4.0,0.37,0.0,0.0,0.0,Toti António Gomes
595,Boubacar Traore,Boubacar,Traoré,20,3,45.0,14.0,0.0,1.4,0.0,...,9.0,2.00,7.62,1.69,0.0,0.00,0.0,0.0,0.0,Boubacar Traoré
596,Cunha,Matheus,Santos Carneiro Da Cunha,20,4,55.0,39.0,6.0,2.3,0.0,...,12.0,1.12,16.76,1.57,5.0,0.47,0.0,0.0,0.0,Matheus Santos Carneiro Da Cunha


In [604]:
# reformat the now_cost variable to sync in with the site values.

players_df['now_cost'] = (players_df['now_cost'] / 10).apply(lambda x: '{:.1f}'.format(x))

In [605]:
cols_to_convert = ['now_cost']
players_df[cols_to_convert] = players_df[cols_to_convert].astype(float)

In [606]:
players_df

,web_name,first_name,second_name,team,element_type,now_cost,total_points,bonus,points_per_game,form,...,goals_conceded,goals_conceded_per_90,expected_goals_conceded,expected_goals_conceded_per_90,clean_sheets,clean_sheets_per_90,saves,saves_per_90,penalties_saved,Player
0,Balogun,Folarin,Balogun,1,4,4.5,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,Folarin Balogun
1,Cédric,Cédric,Alves Soares,1,2,4.0,10.0,0.0,1.2,0.0,...,3.0,1.21,2.83,1.14,0.0,0.00,0.0,0.0,0.0,Cédric Alves Soares
2,M.Elneny,Mohamed,Elneny,1,3,4.5,6.0,0.0,1.2,0.0,...,2.0,1.62,1.29,1.05,0.0,0.00,0.0,0.0,0.0,Mohamed Elneny
3,Fábio Vieira,Fábio,Ferreira Vieira,1,3,5.5,40.0,2.0,1.8,0.0,...,5.0,0.90,5.28,0.95,2.0,0.36,0.0,0.0,0.0,Fábio Ferreira Vieira
4,Gabriel,Gabriel,dos Santos Magalhães,1,2,5.0,146.0,15.0,3.8,0.0,...,43.0,1.14,41.84,1.10,14.0,0.37,0.0,0.0,0.0,Gabriel dos Santos Magalhães
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,N.Semedo,Nélson,Cabral Semedo,20,2,4.5,75.0,5.0,2.1,0.0,...,43.0,1.47,50.02,1.71,8.0,0.27,0.0,0.0,0.0,Nélson Cabral Semedo
594,Toti,Toti António,Gomes,20,2,4.5,43.0,4.0,2.5,0.0,...,14.0,1.29,20.87,1.92,4.0,0.37,0.0,0.0,0.0,Toti António Gomes
595,Boubacar Traore,Boubacar,Traoré,20,3,4.5,14.0,0.0,1.4,0.0,...,9.0,2.00,7.62,1.69,0.0,0.00,0.0,0.0,0.0,Boubacar Traoré
596,Cunha,Matheus,Santos Carneiro Da Cunha,20,4,5.5,39.0,6.0,2.3,0.0,...,12.0,1.12,16.76,1.57,5.0,0.47,0.0,0.0,0.0,Matheus Santos Carneiro Da Cunha


In [607]:
print(players_df.dtypes) 

web_name                              object
first_name                            object
second_name                           object
team                                   int64
element_type                           int64
now_cost                             float64
total_points                         float64
bonus                                float64
points_per_game                      float64
form                                 float64
minutes                              float64
starts                               float64
starts_per_90                        float64
chance_of_playing_next_round         float64
chance_of_playing_this_round          object
selected_by_percent                  float64
transfers_in                         float64
goals_scored                         float64
expected_goals                       float64
expected_goals_per_90                float64
assists                              float64
expected_assists                     float64
expected_a

In [608]:
# Add a calculated columns which will determine ROI per million spent on the player

players_df["ROI_Per_Million"] = (players_df["total_points"] / players_df["now_cost"])


In [609]:
print(players_df.dtypes) 

web_name                              object
first_name                            object
second_name                           object
team                                   int64
element_type                           int64
now_cost                             float64
total_points                         float64
bonus                                float64
points_per_game                      float64
form                                 float64
minutes                              float64
starts                               float64
starts_per_90                        float64
chance_of_playing_next_round         float64
chance_of_playing_this_round          object
selected_by_percent                  float64
transfers_in                         float64
goals_scored                         float64
expected_goals                       float64
expected_goals_per_90                float64
assists                              float64
expected_assists                     float64
expected_a

In [610]:
players_df

,web_name,first_name,second_name,team,element_type,now_cost,total_points,bonus,points_per_game,form,...,goals_conceded_per_90,expected_goals_conceded,expected_goals_conceded_per_90,clean_sheets,clean_sheets_per_90,saves,saves_per_90,penalties_saved,Player,ROI_Per_Million
0,Balogun,Folarin,Balogun,1,4,4.5,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,Folarin Balogun,0.000000
1,Cédric,Cédric,Alves Soares,1,2,4.0,10.0,0.0,1.2,0.0,...,1.21,2.83,1.14,0.0,0.00,0.0,0.0,0.0,Cédric Alves Soares,2.500000
2,M.Elneny,Mohamed,Elneny,1,3,4.5,6.0,0.0,1.2,0.0,...,1.62,1.29,1.05,0.0,0.00,0.0,0.0,0.0,Mohamed Elneny,1.333333
3,Fábio Vieira,Fábio,Ferreira Vieira,1,3,5.5,40.0,2.0,1.8,0.0,...,0.90,5.28,0.95,2.0,0.36,0.0,0.0,0.0,Fábio Ferreira Vieira,7.272727
4,Gabriel,Gabriel,dos Santos Magalhães,1,2,5.0,146.0,15.0,3.8,0.0,...,1.14,41.84,1.10,14.0,0.37,0.0,0.0,0.0,Gabriel dos Santos Magalhães,29.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,N.Semedo,Nélson,Cabral Semedo,20,2,4.5,75.0,5.0,2.1,0.0,...,1.47,50.02,1.71,8.0,0.27,0.0,0.0,0.0,Nélson Cabral Semedo,16.666667
594,Toti,Toti António,Gomes,20,2,4.5,43.0,4.0,2.5,0.0,...,1.29,20.87,1.92,4.0,0.37,0.0,0.0,0.0,Toti António Gomes,9.555556
595,Boubacar Traore,Boubacar,Traoré,20,3,4.5,14.0,0.0,1.4,0.0,...,2.00,7.62,1.69,0.0,0.00,0.0,0.0,0.0,Boubacar Traoré,3.111111
596,Cunha,Matheus,Santos Carneiro Da Cunha,20,4,5.5,39.0,6.0,2.3,0.0,...,1.12,16.76,1.57,5.0,0.47,0.0,0.0,0.0,Matheus Santos Carneiro Da Cunha,7.090909


In [611]:
# Mapping the team and position names can be done using the map() method

players_df['team'] = players_df.team.map(teams_df.set_index('id').name)
players_df['position'] = players_df.element_type.map(elements_types_df.set_index('id').singular_name)

# Filter out players who have no minutes.

players_df = players_df[players_df['minutes'] != 0]

In [612]:
players_df

,web_name,first_name,second_name,team,element_type,now_cost,total_points,bonus,points_per_game,form,...,expected_goals_conceded,expected_goals_conceded_per_90,clean_sheets,clean_sheets_per_90,saves,saves_per_90,penalties_saved,Player,ROI_Per_Million,position
1,Cédric,Cédric,Alves Soares,Arsenal,2,4.0,10.0,0.0,1.2,0.0,...,2.83,1.14,0.0,0.00,0.0,0.0,0.0,Cédric Alves Soares,2.500000,Defender
2,M.Elneny,Mohamed,Elneny,Arsenal,3,4.5,6.0,0.0,1.2,0.0,...,1.29,1.05,0.0,0.00,0.0,0.0,0.0,Mohamed Elneny,1.333333,Midfielder
3,Fábio Vieira,Fábio,Ferreira Vieira,Arsenal,3,5.5,40.0,2.0,1.8,0.0,...,5.28,0.95,2.0,0.36,0.0,0.0,0.0,Fábio Ferreira Vieira,7.272727,Midfielder
4,Gabriel,Gabriel,dos Santos Magalhães,Arsenal,2,5.0,146.0,15.0,3.8,0.0,...,41.84,1.10,14.0,0.37,0.0,0.0,0.0,Gabriel dos Santos Magalhães,29.200000,Defender
5,Havertz,Kai,Havertz,Arsenal,3,7.5,102.0,13.0,2.9,0.0,...,38.25,1.34,9.0,0.32,0.0,0.0,0.0,Kai Havertz,13.600000,Midfielder
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,N.Semedo,Nélson,Cabral Semedo,Wolves,2,4.5,75.0,5.0,2.1,0.0,...,50.02,1.71,8.0,0.27,0.0,0.0,0.0,Nélson Cabral Semedo,16.666667,Defender
594,Toti,Toti António,Gomes,Wolves,2,4.5,43.0,4.0,2.5,0.0,...,20.87,1.92,4.0,0.37,0.0,0.0,0.0,Toti António Gomes,9.555556,Defender
595,Boubacar Traore,Boubacar,Traoré,Wolves,3,4.5,14.0,0.0,1.4,0.0,...,7.62,1.69,0.0,0.00,0.0,0.0,0.0,Boubacar Traoré,3.111111,Midfielder
596,Cunha,Matheus,Santos Carneiro Da Cunha,Wolves,4,5.5,39.0,6.0,2.3,0.0,...,16.76,1.57,5.0,0.47,0.0,0.0,0.0,Matheus Santos Carneiro Da Cunha,7.090909,Forward


In [613]:
# Aggregate the team totals

team_stats = players_df.groupby('team').agg({'total_points': 'sum', 'bonus': 'sum', 'goals_scored': 'sum','assists': 'sum','expected_goals': 'sum','expected_assists': 'sum'}).sort_values('total_points',ascending=False).reset_index()

team_stats['performance_xG'] = team_stats['goals_scored'] - team_stats['expected_goals']
team_stats['performance_xA'] = team_stats['assists'] - team_stats['expected_assists']

columns = ['team_name','team_total_points','team_bonus','team_goals_scored','team_assists','team_expected_goals','team_expected_assists','team_performance_xG','team_performance_xA']

# rename these columns to avoid clashing with merge
team_stats.columns = columns

# Rearrange the columns to make more practical sense
team_stats = team_stats[['team_name','team_total_points','team_bonus','team_goals_scored','team_expected_goals','team_performance_xG','team_assists','team_expected_assists','team_performance_xA']]

# Rename columns
team_stats.rename(columns = {'team_name':'Team', 'team_total_points':'Total Points', 'team_bonus':'Bonus Points', 'team_goals_scored':'Goals', 'team_expected_goals':'xG', 'team_performance_xG':'Goals - xG', 'team_assists':'Assists', 'team_expected_assists':'xA', 'team_performance_xA':'Assists - xA'}, inplace = True)


In [614]:
team_stats

,Team,Total Points,Bonus Points,Goals,xG,Goals - xG,Assists,xA,Assists - xA
0,Arsenal,2407.0,214.0,104.0,93.99,10.01,82.0,53.93,28.07
1,Man City,1972.0,188.0,85.0,72.74,12.26,81.0,54.88,26.12
2,Newcastle,1921.0,170.0,65.0,74.81,-9.81,57.0,44.69,12.31
3,Liverpool,1788.0,148.0,67.0,76.17,-9.17,62.0,52.25,9.75
4,Man Utd,1770.0,165.0,58.0,66.06,-8.06,52.0,44.63,7.37
5,Brentford,1735.0,138.0,55.0,58.17,-3.17,50.0,30.42,19.58
6,Spurs,1728.0,136.0,79.0,63.99,15.01,65.0,38.64,26.36
7,Aston Villa,1578.0,131.0,45.0,47.91,-2.91,48.0,32.96,15.04
8,Brighton,1529.0,120.0,50.0,56.10,-6.10,56.0,38.59,17.41
9,Fulham,1400.0,109.0,46.0,43.41,2.59,40.0,21.13,18.87


In [615]:
# Subset the data based on position

defenders_df = players_df[players_df['position'] == 'Defender']
midfielders_df = players_df[players_df['position'] == 'Midfielder']
forwards_df = players_df[players_df['position'] == 'Forward']
goalkeepers_df = players_df[players_df['position'] == 'Goalkeeper']


In [616]:
# Subset each position dataframe with specific metrics

goalkeepers_df = goalkeepers_df[['Player','team','position','total_points','bonus','points_per_game','form','minutes','now_cost','ROI_Per_Million','goals_conceded','expected_goals_conceded','clean_sheets','saves','saves_per_90']]
goalkeepers_df['performance_xG_def'] = goalkeepers_df['goals_conceded'] - goalkeepers_df['expected_goals_conceded']
# Position newly-calculated metric within the dataframe
goalkeepers_df = goalkeepers_df[['Player','team','position','total_points','bonus','points_per_game','form','minutes','now_cost','ROI_Per_Million','goals_conceded','expected_goals_conceded','performance_xG_def','clean_sheets','saves','saves_per_90']]

defenders_df = defenders_df[['Player','team','position','total_points','bonus','points_per_game','form','minutes','now_cost','ROI_Per_Million','goals_scored','expected_goals','expected_goals_per_90','assists','expected_assists','expected_assists_per_90','goals_conceded','expected_goals_conceded','clean_sheets']]
defenders_df['performance_xG_def'] = defenders_df['goals_conceded'] - defenders_df['expected_goals_conceded']
# Position newly-calculated metric within the dataframe
defenders_df = defenders_df[['Player','team','position','total_points','bonus','points_per_game','form','minutes','now_cost','ROI_Per_Million','goals_scored','expected_goals','expected_goals_per_90','assists','expected_assists','expected_assists_per_90','goals_conceded','expected_goals_conceded','performance_xG_def','clean_sheets']]

midfielders_df = midfielders_df[['Player','team','position','total_points','bonus','points_per_game','form','minutes','now_cost','ROI_Per_Million','goals_scored','expected_goals','expected_goals_per_90','assists','expected_assists','expected_assists_per_90','expected_goals_conceded']]
midfielders_df['performance_xG_off'] = midfielders_df['goals_scored'] - midfielders_df['expected_goals']
# Position newly-calculated metric within the dataframe
midfielders_df = midfielders_df[['Player','team','position','total_points','bonus','points_per_game','form','minutes','now_cost','ROI_Per_Million','goals_scored','expected_goals','performance_xG_off','expected_goals_per_90','assists','expected_assists','expected_assists_per_90','expected_goals_conceded']]

forwards_df = forwards_df[['Player','team','position','total_points','bonus','points_per_game','form','minutes','now_cost','ROI_Per_Million','goals_scored','expected_goals','expected_goals_per_90','assists','expected_assists','expected_assists_per_90']]
forwards_df['performance_xG_off'] = forwards_df['goals_scored'] - forwards_df['expected_goals']
# Position newly-calculated metric within the dataframe
forwards_df = forwards_df[['Player','team','position','total_points','bonus','points_per_game','form','minutes','now_cost','ROI_Per_Million','goals_scored','expected_goals','performance_xG_off','expected_goals_per_90','assists','expected_assists','expected_assists_per_90']]



In [617]:
goalkeepers_df.rename(columns = {'Player':'player', 'team':'team', 'position':'position', 'total_points':'points', 'bonus':'bonus', 'points_per_game':'points p/g', 'form':'form', 'minutes':'minutes', 'now_cost':'now cost', 'ROI_Per_Million':'roi per million', 'goals_conceded':'conceded', 'expected_goals_conceded':'xGA', 'performance_xG_def':'conceded - xGA', 'clean_sheets':'clean sheets', 'saves':'saves', 'saves_per_90':'saves per 90'}, inplace = True)

In [618]:
defenders_df.rename(columns = {'Player':'player', 'team':'team', 'position':'position', 'total_points':'points', 'bonus':'bonus', 'points_per_game':'points p/g', 'form':'form', 'minutes':'minutes', 'now_cost':'now cost', 'ROI_Per_Million':'roi per million', 'goals_scored':'goals', 'expected_goals':'xG', 'expected_goals_per_90':'xG per 90', 'assists':'assists', 'expected_assists':'xA', 'expected_assists_per_90':'xA per 90', 'expected_goal_involvements':'expected goal involvements', 'expected_goal_involvements_per_90':'expected goal involvement per 90', 'goals_conceded':'conceded', 'expected_goals_conceded':'xGA', 'performance_xG_def':'conceded - xGA', 'clean_sheets':'clean sheets'}, inplace = True)

In [619]:
midfielders_df.rename(columns = {'Player':'player', 'team':'team', 'position':'position', 'total_points':'points', 'bonus':'bonus', 'points_per_game':'points p/g', 'form':'form', 'minutes':'minutes', 'now_cost':'now cost', 'ROI_Per_Million':'roi per million', 'goals_scored':'goals', 'expected_goals':'xG', 'performance_xG_off':'goals - xG', 'expected_goals_per_90':'xG per 90', 'assists':'assists', 'expected_assists':'xA', 'expected_assists_per_90':'xA per 90', 'expected_goals_conceded':'xGA'}, inplace = True)

In [620]:
forwards_df.rename(columns = {'Player':'player', 'team':'team', 'position':'position', 'total_points':'points', 'bonus':'bonus', 'points_per_game':'points p/g', 'form':'form', 'minutes':'minutes', 'now_cost':'now cost', 'ROI_Per_Million':'roi per million', 'goals_scored':'goals', 'expected_goals':'xG', 'performance_xG_off':'goals - xG', 'expected_goals_per_90':'xG per 90', 'assists':'assists', 'expected_assists':'xA', 'expected_assists_per_90':'xA per 90'}, inplace = True)

In [621]:
forwards_df


,player,team,position,points,bonus,points p/g,form,minutes,now cost,roi per million,goals,xG,goals - xG,xG per 90,assists,xA,xA per 90
7,Gabriel Fernando de Jesus,Arsenal,Forward,125.0,17.0,4.8,0.0,2064.0,8.0,15.625000,11.0,14.24,-3.24,0.62,7.0,2.91,0.13
12,Eddie Nketiah,Arsenal,Forward,63.0,5.0,2.1,0.0,1071.0,5.5,11.454545,4.0,7.89,-3.89,0.66,2.0,1.26,0.11
35,Cameron Archer,Aston Villa,Forward,6.0,0.0,1.0,0.0,43.0,4.5,1.333333,0.0,0.00,0.00,0.00,0.0,0.01,0.02
46,Jhon Durán,Aston Villa,Forward,11.0,0.0,0.9,0.0,126.0,5.0,2.200000,0.0,0.59,-0.59,0.42,0.0,0.02,0.01
61,Ollie Watkins,Aston Villa,Forward,175.0,25.0,4.7,0.0,3129.0,8.0,21.875000,15.0,16.15,-1.15,0.46,8.0,2.66,0.08
78,Kieffer Moore,Bournemouth,Forward,57.0,3.0,2.1,0.0,1269.0,5.0,11.400000,4.0,4.18,-0.18,0.30,0.0,0.50,0.04
84,Antoine Semenyo,Bournemouth,Forward,18.0,1.0,1.6,0.0,250.0,4.5,4.000000,1.0,0.36,0.64,0.13,0.0,0.02,0.01
87,Dominic Solanke,Bournemouth,Forward,130.0,14.0,3.9,0.0,2865.0,6.5,20.000000,6.0,9.09,-3.09,0.29,10.0,2.30,0.07
103,Halil Dervişoğlu,Brentford,Forward,4.0,0.0,4.0,0.0,6.0,4.5,0.888889,0.0,0.03,-0.03,0.45,1.0,0.00,0.00
120,Ivan Toney,Brentford,Forward,182.0,35.0,5.5,0.0,2953.0,8.0,22.750000,20.0,18.74,1.26,0.57,4.0,3.72,0.11


In [622]:
# Export each dataframe to a csv
team_stats.to_csv('fpldata/teams.csv',index=False)
goalkeepers_df.to_csv('fpldata/goalkeepers.csv',index=False)
defenders_df.to_csv('fpldata/defenders.csv',index=False)
midfielders_df.to_csv('fpldata/midfielders.csv',index=False)
forwards_df.to_csv('fpldata/forwards.csv',index=False)